<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Natural Language Processing (NLP)
## *Data Science Unit 4 Sprint 1 Assignment 1*

Your goal in assignment: find the attributes of the best & worst coffee shops in the dataset. The text is fairly raw: dates in the review, extra words in the `star_rating` column, etc. You'll probably want to clean that stuff up for a better analysis. 

Analyze the corpus of text using text visualizations of token frequency. Try cleaning the data as much as possible. Try the following techniques: 
- Lemmatization
- Custom stopword removal

Keep in mind the attributes of good tokens. Once you have a solid baseline, layer in the star rating in your visualization(s). Keep part in this assignment - produce a write-up of the attributes of the best and worst coffee shops. Based on your analysis, what makes the best the best and the worst the worst. Use graphs and numbesr from your analysis to support your conclusions. There should be plenty of markdown cells! :coffee:

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

In [ ]:
# setting up natural language processor with tokenizer
nlp = spacy.load('en_core_web_lg')
tokenizer = Tokenizer(nlp.vocab)

In [3]:
url = "https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/master/module1-text-data/data/yelp_coffeeshop_review_data.csv"

shops = pd.read_csv(url)
shops.head()

,coffee_shop_name,full_review_text,star_rating
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the atm...,5.0 star rating
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, Ambia...",4.0 star rating
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in Brunch Spots ...,4.0 star rating
3,The Factory - Cafe With a Soul,11/25/2016 Very cool decor! Good drinks Nice ...,2.0 star rating
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are located within ...,4.0 star rating


In [63]:
# copies dataframe to new df
df = shops.copy()

# renames columns for ease of use later
df.columns=['shop', 'text', 'rating']

# changes rating column to have only the number (still as a string)
df['rating'] = df['rating'].str[1:4]

# converts number string to float, then float to int
df['rating'] = df['rating'].astype(float)
df['rating'] = df['rating'].astype(int)

# creates a date column
df['date'] = df['text'].str.split(' ').apply(lambda x: x[1])

# converts data column to datetime dtype
df['date'] = df['date'].apply(pd.to_datetime)

# creates a new column with tokenized data using lecture function
df['counter'] = df['text'].apply(tokenizer)

# takes every token past the date token
df['counter'] = df['counter'].apply(lambda x: x[2:])

# converts list to counter object
df['counter'] = df['counter'].apply(Counter)

In [64]:
df.head()

,shop,text,rating,date,counter
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the atm...,5,2016-11-25,"{1: 1, check-in: 1, Love: 1, love: 1, loved: 1..."
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, Ambia...",4,2016-12-02,"{Listed: 1, in: 1, Date: 1, Night:: 1, Austin,..."
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in Brunch Spots ...,4,2016-11-30,"{1: 1, check-in: 1, Listed: 1, in: 1, Brunch: ..."
3,The Factory - Cafe With a Soul,11/25/2016 Very cool decor! Good drinks Nice ...,2,2016-11-25,"{Very: 1, cool: 1, decor!: 1, Good: 1, drinks:..."
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are located within ...,4,2016-12-03,"{1: 1, check-in: 1, They: 1, are: 1, located: ..."


## How do we want to analyze these coffee shop tokens? 

- Overall Word / Token Count
- View Counts by Rating 
- *Hint:* a 'bad' coffee shops has a rating betweeen 1 & 3 based on the distribution of ratings. A 'good' coffee shop is a 4 or 5. 

In [43]:
# sums the rows into one counter
wordCount = df['counter'].sum()

In [39]:
# looks at most common words across all reviews

wordCount.most_common(10)

[('the', 34809),
 ('and', 26650),
 ('a', 22755),
 ('i', 20237),
 ('to', 17164),
 ('of', 12600),
 ('is', 11999),
 ('coffee', 10353),
 ('was', 9707),
 ('in', 9546)]

In [45]:
# creates dataframes for good and bad ratings
dfGood = df[df['rating'] > 3]
dfBad = df[df['rating'] < 4]

# makes sure the size of the new dataframes add up to the size of the original dataframe
assert len(df) == len(dfGood) + len(dfBad)

In [46]:
# creates counters for good/bad reivews
wordCountGood = dfGood['counter'].sum()
wordCountBad = dfBad['counter'].sum()

In [47]:
# looks at top 10 words used in good reviews
wordCountGood.most_common(10)

[('the', 26616),
 ('and', 21311),
 ('a', 17706),
 ('i', 14952),
 ('to', 12763),
 ('of', 9932),
 ('is', 9644),
 ('coffee', 8234),
 ('in', 7517),
 ('it', 6898)]

In [48]:
# looks at top 10 words in bad reviews
wordCountBad.most_common(10)

[('the', 8193),
 ('and', 5339),
 ('i', 5285),
 ('a', 5049),
 ('to', 4401),
 ('was', 2933),
 ('of', 2668),
 ('it', 2481),
 ('is', 2355),
 ('coffee', 2119)]

## Can visualize the words with the greatest difference in counts between 'good' & 'bad'?

Couple Notes: 
- Rel. freq. instead of absolute counts b/c of different numbers of reviews
- Only look at the top 5-10 words with the greatest differences


In [50]:
wordCountBad.most_common(normalize=True)

TypeError: most_common() got an unexpected keyword argument 'normalize'

## Stretch Goals

* Analyze another corpus of documents - such as Indeed.com job listings ;).
* Play the the Spacy API to
 - Extract Named Entities
 - Extracting 'noun chunks'
 - Attempt Document Classification with just Spacy
 - *Note:* This [course](https://course.spacy.io/) will be of interesting in helping you with these stretch goals. 
* Try to build a plotly dash app with your text data 

